In [11]:
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential,load_model,Model
from keras.layers import LSTM,Dense,Input,Flatten,Concatenate
from keras.layers import LSTMCell,RNN,Layer
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
#sess =tf.Session()

In [13]:
LSTMModel = load_model('./models/25hz/realtime_LSTM_v2_Tap_32_3Cepoch_450loss_0.08.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [14]:
weights=LSTMModel.get_weights()

In [7]:
for i in range(len(weights)):
    print(weights[i].shape)

(3, 128)
(32, 128)
(128,)
(32, 1)
(1,)


In [23]:
np.savez('weight.npz',weights)

In [25]:
weights_load=np.load('weight.npz')

In [40]:
weights_load.files

['arr_0']

In [42]:
print(len(weights_load[weights_load.files[0]]))

5


In [39]:
for i in range(len(weights_load['arr_0'])):
    print(weights_load['arr_0'][i].shape)

(3, 128)
(32, 128)
(128,)
(32, 1)
(1,)


In [8]:
from keras.initializers import Constant

In [9]:
class LSTMLayer(Layer):
    def __init__(self,units=32,weights):
        self.units=units
        self.weights=weights
        
        self.weight_b=weights[2]
        self.weight_dw=weights[3]
        self.weight_db=weights[4]
        
        super(LSTMLayer,self).__init__(**kwargs)
    def build(self,input_shape):
        self.wi=self.add_weight('wi',shape = (input_shape[-1],self.units)initializer=Constant(self.weights[0][:,0:self.units]))
        self.wf=self.add_weight('wf',shape = (input_shape[-1],self.units)initializer=Constant(self.weights[0][:,self.units:self.units*2]))
        self.wc=self.add_weight('wc',shape = (input_shape[-1],self.units)initializer=Constant(self.weights[0][:,self.units*2:self.units*3]))
        self.wo=self.add_weight('wo',shape = (input_shape[-1],self.units)initializer=Constant(self.weights[0][:,self.units*3:self.units*4]))
        self.ui=self.add_weight('ui',shape = (self.units,self.units)initializer=Constant(self.weights[1][:,0:self.units]))
        self.uf=self.add_weight('uf',shape = (self.units,self.units)initializer=Constant(self.weights[1][:,self.units:self.units*2]))
        self.uc=self.add_weight('uc',shape = (self.units,self.units)initializer=Constant(self.weights[1][:,self.units*2:self.units*3]))
        self.uo=self.add_weight('uo',shape = (self.units,self.units)initializer=Constant(self.weights[1][:,self.units*3:self.units*4]))
        
        self.bi=self.add_weight('bi',shape = (self.units,)initializer=Constant(self.weights[2][0:self.units]))
        self.bf=self.add_weight('bf',shape = (self.units,)initializer=Constant(self.weights[2][self.units:self.units*2]))
        self.bc=self.add_weight('bc',shape = (self.units,)initializer=Constant(self.weights[2][self.units*2:self.units*3]))
        self.bo=self.add_weight('bo',shape = (self.units,)initializer=Constant(self.weights[2][self.units*3:self.units*4]))
        super(LSTMLayer, self).build(input_shape)
        
    def call(self,x):
        
        
        
        
        
        
        return
    def compute_output_shape(self,input_shape):
        return(input_shape[0],1)
        
    

SyntaxError: non-default argument follows default argument (<ipython-input-9-7de7b6314e9f>, line 2)

In [2]:
class MyLayer(Layer):

    def __init__(self, output_dim=(1), **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[-1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [3]:
def build_model(units):
    myInput = Input(shape=(1,13,))
    output=MyLayer(output_dim = (1))(myInput)
    return Model(inputs=myInput,outputs=output)

In [4]:
myModel=build_model(16)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
myModel.save('myModel.h5')

In [6]:
loaded_model=load_model('myModel.h5',custom_objects={'MyLayer':MyLayer})

C:\Users\Guo\Anaconda3\envs\tf_1_13_cpu\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
#saved_model_dir='realtime_LSTM_32U_3C.hdf5'
saved_model_dir='myModel.h5'
frozen_graph = "realtime_LSTM_v2_Tap_32_3Cepoch_450loss_0.08.h5.pb"
#frozen_graph = "blinknn_frozen_model.pb"
#converter=tf.lite.TFLiteConverter.from_keras_model_file(model_file=saved_model_dir,custom_objects={'MyLayer':MyLayer})
converter=tf.lite.TFLiteConverter.from_frozen_graph(frozen_graph,input_arrays={'input_1'},output_arrays={'output_node0'})
tflite_model = converter.convert()
open("realLSTM32U3C.tflite", "wb").write(tflite_model)

ConverterError: TOCO failed. See console for info.
2019-06-25 11:14:14.583406: I tensorflow/lite/toco/import_tensorflow.cc:1324] Converting unsupported operation: TensorArrayV3
2019-06-25 11:14:14.584064: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "CPU"') for unknown op: WrapDatasetVariant
2019-06-25 11:14:14.584408: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknown op: WrapDatasetVariant
2019-06-25 11:14:14.584874: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "CPU"') for unknown op: UnwrapDatasetVariant
2019-06-25 11:14:14.585222: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknown op: UnwrapDatasetVariant
2019-06-25 11:14: